<hr style="border:2px solid gray">

### Author  : SIVA VIKNESH 
### Email   : siva.viknesh@sci.utah.edu / sivaviknesh14@gmail.com 
### Address : SCI INSTITUTE, UNIVERSITY OF UTAH, SALT LAKE CITY, UTAH, USA 
<hr style="border:2px solid gray">

In [ ]:
import os
import torch
import math
import torch.nn as nn
import torch.optim as optim
from torch.nn.parameter import Parameter
import numpy as np
import matplotlib.pyplot as plt
import scipy
import vtk
from vtk.util import numpy_support as VN
from itertools import combinations

Here, we consider a unsteady flow inside a symmetric stenosed coronary artery computed by imposing time-varying inlet flow rate using an open sourced - SimVascular FEM fluid solver. The post-stenoic velocity flow field over a grid of 512X512 is used for the project work. 

In [ ]:
x_dim = 512
y_dim = 512
data_file = "data_"
directory = os.getcwd()  # GET THE CURRENT WORKING DIRECTORY  
path      = directory + '/'
data_file = path + data_file
Nfiles    = 100

In [ ]:
fieldname  = 'Vmag'
Vel_data   = np.zeros((x_dim*y_dim, Nfiles))

for i in range(Nfiles):
    file_name = data_file + str(i) + ".vtk"
    print ('READING THE DATA FILE: ', file_name[len(directory)+1:])
    reader = vtk.vtkStructuredGridReader()
    reader.SetFileName(file_name)
    reader.Update()
    data = reader.GetOutput()
    pointData = data.GetPointData().GetArray(fieldname)
    velocity  = np.reshape(VN.vtk_to_numpy(pointData), (1, x_dim*y_dim))
    Vel_data [:, i] = (2.0*velocity - np.min(velocity) - np.max(velocity))/ (np.max(velocity) - np.min(velocity))
    print ("*"*85)


First, we normalize the temporal velocity data by its spatially averaged mean value at a given time instant. Then, the matrix size is transformed to a square matrix, thereby computing the SVD operation more efficiently.

In [ ]:
U, E, V  = np.linalg.svd(Vel_data, full_matrices=False)
Eigen_val = np.sqrt(E)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(12, 3))
fig.suptitle('Singular Values Spectrum')

ax[0].plot(Eigen_val, '-o')
ax[0].set(xlabel="Modes",ylabel="Singular Values")

ax[1].plot(Eigen_val, 'o')
ax[1].set(xlabel="Modes",ylabel="Singular Values")
ax[1].set_yscale('log')

ax[2].plot(np.cumsum(Eigen_val)/np.sum(Eigen_val), '-o')
ax[2].set(xlabel="Modes",ylabel="Cumulative Energy")
fig.subplots_adjust(wspace=0.5)

In [ ]:
mode1 = np.reshape(U[:, 0], (x_dim, y_dim))
mode2 = np.reshape(U[:, 1], (x_dim, y_dim))
mode3 = np.reshape(U[:, 2], (x_dim, y_dim))

coeff1 = V [0, :]
coeff2 = V [1, :]
coeff3 = V [2, :]
coeff4 = V [3, :]

fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(12, 3))
fig.suptitle('Spatial Modes')

ax[0].contourf(mode1) 
ax[0].set_title('Mode 1')
ax[0].axis('off')

ax[1].contourf(mode2) 
ax[1].set_title('Mode 2')
ax[1].axis('off')

ax[2].contourf(mode3) 
ax[2].set_title('Mode 3')
ax[2].axis('off')
fig.subplots_adjust(top=0.8)

fig, ax = plt.subplots(nrows = 1, ncols = 4, figsize =(12, 3))
fig.suptitle('Temporal Modes')

ax[0].plot(coeff1) 
ax[0].set_title('Mode 1')

ax[1].plot(coeff2)
ax[1].set_title('Mode 2')


ax[2].plot(coeff3)
ax[2].set_title('Mode 3')

ax[3].plot(coeff4)
ax[3].set_title('Mode 4')

fig.subplots_adjust(top=0.8)



fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(14, 4))
fig.suptitle('Mode Pair Trajectories')

ax[0].plot(coeff1, coeff2) 
ax[0].set_title('Mode 1')
ax[0].set_xlabel('$\mathregular{a_1}$')
ax[0].set_ylabel('$\mathregular{a_2}$')

ax[1].plot(coeff1, coeff3)
ax[1].set_title('Mode 2')
ax[1].set_xlabel('$\mathregular{a_1}$')
ax[1].set_ylabel('$\mathregular{a_3}$')

ax[2].plot(coeff1, coeff4)
ax[2].set_title('Mode 3')
ax[2].set_xlabel('$\mathregular{a_1}$')
ax[2].set_ylabel('$\mathregular{a_4}$')
fig.subplots_adjust(top=0.8)

SINDy computation starts from here, utilizing the computed PCA modes.

In [ ]:
processor = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("AVAILABLE PROCESSOR:", processor, '\n')

N_modes     = 3
modes       = torch.Tensor(V [:N_modes, :]).to(processor)

# HYPERPARAMETERS FOR THE SINDy POD METHODOLOGY
Epochs        = 15000
learning_rate = 1e-3
step_epoch    = 3500
decay_rate    = 0.50

N_terms = math.comb(N_modes, 1) + math.comb(N_modes, 2) + math.comb(N_modes, 3) + N_modes

# COMPUTING THE COMBINATIONS AMONG THE THREE CHOSEN TEMPORAL MODES
A1 = modes [0, :].reshape(1, -1)
A2 = modes [1, :].reshape(1, -1)
A3 = modes [2, :].reshape(1, -1)

A1A2   = (A1*A2).reshape(1, -1) 
A2A3   = (A2*A3).reshape(1, -1)
A3A1   = (A3*A1).reshape(1, -1)

A11A2  = (A1*A1*A2).reshape(1, -1) 
A22A3  = (A2*A2*A3).reshape(1, -1)
A33A1  = (A3*A3*A1).reshape(1, -1)

A1A22  = (A1*A2*A2).reshape(1, -1) 
A2A33  = (A2*A3*A3).reshape(1, -1)
A3A11  = (A3*A1*A1).reshape(1, -1)

A1A2A3 = (A1*A2*A3).reshape(1, -1)

A11A2A3 = (A1*A1*A2*A3).reshape(1, -1)
A1A22A3 = (A1*A2*A2*A3).reshape(1, -1)
A1A2A33 = (A1*A2*A3*A3).reshape(1, -1)

A11A22A3 = (A1*A1*A2*A2*A3).reshape(1, -1)
A1A22A33 = (A1*A2*A2*A3*A3).reshape(1, -1)
A11A2A33 = (A1*A1*A2*A3*A3).reshape(1, -1)

A_candidates = torch.transpose(torch.vstack ((torch.ones_like(A1), A1, A2, A3, A1A2, A2A3, A3A1, A11A2, A22A3, A33A1, A1A22, A2A33, A3A11, A1A2A3,
                              A11A2A3, A1A22A3, A1A2A33, A11A22A3, A1A22A33, A11A2A33)), 0, 1)
print(A_candidates.shape)                             


In [ ]:
def DERIVATIVE(a, Nfiles):
    dadt = torch.zeros_like(a)
    for i in range(Nfiles-1):
        dadt [:, i] = a[:, i+1] - a[:, i]
    dadt [:, -1] = a[:, -2] - a[:, -1]
    return dadt

In [ ]:
class SINDy_MODEL(nn.Module):
    def __init__(self, a):
        super().__init__()    
        self.a  = a
    def forward(self, x):
        output = x @ self.a
        return output

In [ ]:
# AMPLITUDE COEFFICIENTS OF SINDy MODEL

# NO Lambda
COEFF_ADT1 = torch.ones(A_candidates.shape[1], N_modes, requires_grad= True, device= processor)

# Static Scalar Lambda
COEFF_ADT2 = torch.ones(A_candidates.shape[1], N_modes, requires_grad= True, device= processor)

# Dynamic Scalar Lambda
COEFF_ADT3 = torch.ones(A_candidates.shape[1], N_modes, requires_grad= True, device= processor)

# Dynamic matrix Lambda
COEFF_ADT4 = torch.ones(A_candidates.shape[1], N_modes, requires_grad= True, device= processor)

In [ ]:
# COEFFICIENTS OF SINDy MODEL
optim_COEFF_ADT1 = optim.Adam([COEFF_ADT1],  lr=learning_rate, betas = (0.9,0.99),eps = 10**-15)
optim_COEFF_ADT2 = optim.Adam([COEFF_ADT2],  lr=learning_rate, betas = (0.9,0.99),eps = 10**-15)
optim_COEFF_ADT3 = optim.Adam([COEFF_ADT3],  lr=learning_rate, betas = (0.9,0.99),eps = 10**-15)
optim_COEFF_ADT4 = optim.Adam([COEFF_ADT4],  lr=learning_rate, betas = (0.9,0.99),eps = 10**-15)

# WEIGHT FUNCTION OF SINDy MODEL
W_FUNCTION2   = Parameter(torch.tensor(0.001), requires_grad= True)
W_FUNCTION3   = Parameter(torch.tensor(0.001), requires_grad= True)
W_FUNCTION4   = Parameter(torch.ones_like(COEFF_ADT4), requires_grad= True)

optim_Lambda3  = optim.Adam([W_FUNCTION3], lr = learning_rate, betas = (0.9,0.99),eps = 10**-15)
optim_Lambda4  = optim.Adam([W_FUNCTION4], lr = learning_rate, betas = (0.9,0.99),eps = 10**-15)

# STEP DECAY DYNAMIC LEARNING RATE
scheduler_COEFF_ADT1 = torch.optim.lr_scheduler.StepLR(optim_COEFF_ADT1, step_size=step_epoch, gamma=decay_rate)
scheduler_COEFF_ADT2 = torch.optim.lr_scheduler.StepLR(optim_COEFF_ADT2, step_size=step_epoch, gamma=decay_rate)
scheduler_COEFF_ADT3 = torch.optim.lr_scheduler.StepLR(optim_COEFF_ADT3, step_size=step_epoch, gamma=decay_rate)
scheduler_COEFF_ADT4 = torch.optim.lr_scheduler.StepLR(optim_COEFF_ADT4, step_size=step_epoch, gamma=decay_rate)

scheduler_LAMBDA3    = torch.optim.lr_scheduler.StepLR(optim_Lambda3,    step_size=step_epoch, gamma=decay_rate)
scheduler_LAMBDA4    = torch.optim.lr_scheduler.StepLR(optim_Lambda4,    step_size=step_epoch, gamma=decay_rate)

**CASE I: SINDy with $\lambda = 0$**

In [ ]:
A1A2A3_time_deriv = DERIVATIVE (modes, Nfiles)

# TEMPORAL MODE 1
A1_DT = SINDy_MODEL(COEFF_ADT1[:, 0]).to(processor)

# TEMPORAL MODE 2
A2_DT = SINDy_MODEL(COEFF_ADT1[:, 1]).to(processor)

# TEMPORAL MODE 3
A3_DT = SINDy_MODEL(COEFF_ADT1[:, 2]).to(processor)

Loss_data1     = torch.empty(size=(Epochs, 1))
loss_function = nn.MSELoss()

for epoch in range(Epochs):
    A1_out, A2_out, A3_out  = A1_DT (A_candidates), A2_DT (A_candidates), A3_DT (A_candidates)
    output_data  = torch.vstack((A1_out , A2_out, A3_out))
    loss_epoch   = loss_function (A1A2A3_time_deriv, output_data)
    
    optim_COEFF_ADT1.zero_grad()
    loss_epoch.backward()

    with torch.no_grad():
        optim_COEFF_ADT1.step()       
        Loss_data1 [epoch] = loss_epoch.detach()
        
    print('LOSS DATA, [EPOCH =', epoch,  ']:',  Loss_data1 [epoch].item())
    print('LEARNING RATE:', optim_COEFF_ADT1.param_groups[0]['lr'])
    print ("*"*85)
       
    scheduler_COEFF_ADT1.step()


**CASE 2: SINDy with $\lambda = 0.1$. In this case, $\lambda$ is chosen to be fixed during training process.**

In [ ]:
# TEMPORAL MODE 1
A1_DT = SINDy_MODEL(COEFF_ADT2 [:, 0]).to(processor)

# TEMPORAL MODE 2
A2_DT = SINDy_MODEL(COEFF_ADT2 [:, 1]).to(processor)

# TEMPORAL MODE 3
A3_DT = SINDy_MODEL(COEFF_ADT2 [:, 2]).to(processor)

Loss_data2    = torch.empty(size=(Epochs, 1))
loss_function = nn.MSELoss()

for epoch in range(Epochs):
    A1_out, A2_out, A3_out  = A1_DT (A_candidates), A2_DT (A_candidates), A3_DT (A_candidates)
    output_data  = torch.vstack((A1_out , A2_out, A3_out))
    loss_epoch   = loss_function (A1A2A3_time_deriv, output_data) + torch.linalg.matrix_norm(W_FUNCTION2*COEFF_ADT2, ord =1)
    
    optim_COEFF_ADT2.zero_grad()    
    loss_epoch.backward()

    with torch.no_grad():
        optim_COEFF_ADT2.step()      
        Loss_data2 [epoch] = loss_epoch.detach()
        
    print('LOSS DATA, [EPOCH =', epoch,  ']:',  Loss_data2 [epoch].item())
    print('LEARNING RATE:', optim_COEFF_ADT1.param_groups[0]['lr'])
    print ("*"*85)
       
    scheduler_COEFF_ADT2.step()

**CASE 3: SINDy with  dynamic scalar $\lambda$. In this case,  the scalar $\lambda$ is optimised  during training process.**

In [ ]:
# TEMPORAL MODE 1
A1_DT = SINDy_MODEL(COEFF_ADT3 [:, 0]).to(processor)

# TEMPORAL MODE 2
A2_DT = SINDy_MODEL(COEFF_ADT3 [:, 1]).to(processor)

# TEMPORAL MODE 3
A3_DT = SINDy_MODEL(COEFF_ADT3 [:, 2]).to(processor)

Loss_data3    = torch.empty(size=(Epochs, 1))
loss_function = nn.MSELoss()

for epoch in range(Epochs):
    A1_out, A2_out, A3_out  = A1_DT (A_candidates), A2_DT (A_candidates), A3_DT (A_candidates)
    output_data  = torch.vstack((A1_out , A2_out, A3_out))
    loss_epoch   = loss_function (A1A2A3_time_deriv, output_data) + torch.linalg.matrix_norm(torch.abs(W_FUNCTION3)*COEFF_ADT3, ord =1)
    
    optim_COEFF_ADT3.zero_grad()
    optim_Lambda3.zero_grad()
    loss_epoch.backward()

    with torch.no_grad():
        optim_COEFF_ADT3.step()    
        optim_Lambda3.step()
        Loss_data3 [epoch] = loss_epoch.detach()
        
    print('LOSS DATA, [EPOCH =', epoch,  ']:',  Loss_data3 [epoch].item())
    print('LEARNING RATE:', optim_COEFF_ADT3.param_groups[0]['lr'])
    print ("*"*85)
       
    scheduler_COEFF_ADT3.step()
    scheduler_LAMBDA3.step()

**CASE 4: SINDy with  dynamic matrix $\lambda $. In this case,  the matrix $\lambda$ is optimised  during training process.**

In [ ]:
# TEMPORAL MODE 1
A1_DT = SINDy_MODEL(COEFF_ADT4 [:, 0]).to(processor)

# TEMPORAL MODE 2
A2_DT = SINDy_MODEL(COEFF_ADT4 [:, 1]).to(processor)

# TEMPORAL MODE 3
A3_DT = SINDy_MODEL(COEFF_ADT4 [:, 2]).to(processor)

Loss_data4     = torch.empty(size=(Epochs, 1))
loss_function = nn.MSELoss()

for epoch in range(Epochs):
    A1_out, A2_out, A3_out  = A1_DT (A_candidates), A2_DT (A_candidates), A3_DT (A_candidates)
    output_data  = torch.vstack((A1_out , A2_out, A3_out))
    loss_epoch   = loss_function (A1A2A3_time_deriv, output_data) + torch.linalg.matrix_norm(torch.abs(W_FUNCTION4)*COEFF_ADT4, ord =1)
    
    optim_COEFF_ADT4.zero_grad()
    optim_Lambda4.zero_grad()
    
    loss_epoch.backward()

    with torch.no_grad():
        optim_COEFF_ADT4.step()
        optim_Lambda4.step()        
        Loss_data4 [epoch] = loss_epoch.detach()
        
    print('LOSS DATA, [EPOCH =', epoch,  ']:',  Loss_data4 [epoch].item())
    print('LEARNING RATE:', optim_COEFF_ADT4.param_groups[0]['lr'])
    print ("*"*85)
       
    scheduler_COEFF_ADT4.step()
    scheduler_LAMBDA4.step()

In [ ]:
plt.plot(Loss_data1.detach().cpu().numpy(), label = "$\lambda = 0$")
plt.plot(Loss_data2.detach().cpu().numpy(), label = "$\lambda = 10^{-2}$")
plt.plot(Loss_data3.detach().cpu().numpy(), label = "$\lambda_{scalar}$")
plt.plot(Loss_data4.detach().cpu().numpy(), label = "$\lambda_{matrix}$")
plt.xlabel('Epoch') 
plt.ylabel('Loss')
plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
threshold = 0.001
#****************************************************************************#
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(12, 3))
fig.suptitle('$\lambda = 0$')

ax[0].plot(COEFF_ADT1 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('Mode 1')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')

ax[1].plot(COEFF_ADT1 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Mode 2')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')

ax[2].plot(COEFF_ADT1 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Mode 3')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
fig.subplots_adjust(top=0.8)
plt.show()

#****************************************************************************#

fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(12, 3))
fig.suptitle('$\lambda = 10^{-2} $')

ax[0].plot(COEFF_ADT2 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('Mode 1')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')

ax[1].plot(COEFF_ADT2 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Mode 2')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')

ax[2].plot(COEFF_ADT2 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Mode 3')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
fig.subplots_adjust(top=0.8)
plt.show()

#****************************************************************************#

fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(12, 3))
fig.suptitle('$\lambda_{scalar} $')

ax[0].plot(COEFF_ADT3 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('Mode 1')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')

ax[1].plot(COEFF_ADT3 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Mode 2')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')

ax[2].plot(COEFF_ADT3 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Mode 3')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
fig.subplots_adjust(top=0.8)
plt.show()
#****************************************************************************#
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(12, 3))
fig.suptitle('$\lambda_{matrix} $')

ax[0].plot(COEFF_ADT4 [:, 0].detach().cpu().numpy(), 'o') 
ax[0].set_title('Mode 1')
ax[0].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[0].axhline(y = -threshold, color = 'r', linestyle = '-')

ax[1].plot(COEFF_ADT4 [:, 1].detach().cpu().numpy(), 'o') 
ax[1].set_title('Mode 2')
ax[1].axhline(y =  threshold, color = 'r', linestyle = '-')
ax[1].axhline(y = -threshold, color = 'r', linestyle = '-')

ax[2].plot(COEFF_ADT4 [:, 2].detach().cpu().numpy(), 'o') 
ax[2].set_title('Mode 3')
ax[2].axhline(y = threshold, color = 'r', linestyle = '-')
ax[2].axhline(y = -threshold, color = 'r', linestyle = '-')
fig.subplots_adjust(top=0.8)
plt.show()


The optimised coefficient matrix is further reduced by assuming the terms to be zero within the chosen threshold (0.001). 

In [ ]:
# TRUNCATED SINDy COEFFICIENTS

# CASE 2: LAMBDA
C1_h1 = torch.clone(COEFF_ADT2[:, 0]).detach()
C1_h1[torch.logical_and(C1_h1>=-threshold, C1_h1<=threshold)] = 0.0

C2_h1 = torch.clone(COEFF_ADT2[:, 1]).detach()
C2_h1[torch.logical_and(C2_h1>=-threshold, C2_h1<=threshold)] = 0.0

C3_h1 = torch.clone(COEFF_ADT2[:, 2]).detach()
C3_h1[torch.logical_and(C3_h1>=-threshold, C3_h1<=threshold)] = 0.0

#****************************************************************************#

# CASE 4: LAMBDA
C1_h2 = torch.clone(COEFF_ADT4[:, 0]).detach()
C1_h2[torch.logical_and(C1_h2>=-threshold, C1_h2<=threshold)] = 0.0

C2_h2 = torch.clone(COEFF_ADT4[:, 1]).detach()
C2_h2[torch.logical_and(C2_h2>=-threshold, C2_h2<=threshold)] = 0.0

C3_h2 = torch.clone(COEFF_ADT4[:, 2]).detach()
C3_h2[torch.logical_and(C3_h2>=-threshold, C3_h2<=threshold)] = 0.0

In [ ]:
A1_DT_Predict1 = A_candidates @ C1_h1
A2_DT_Predict1 = A_candidates @ C2_h1
A3_DT_Predict1 = A_candidates @ C3_h1

A1_DT_Predict2 = A_candidates @ C1_h2
A2_DT_Predict2 = A_candidates @ C2_h2
A3_DT_Predict2 = A_candidates @ C3_h2

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 3, figsize =(15, 4))
fig.suptitle('TEMPORAL POD MODES')

ax[0].plot(A1A2A3_time_deriv[0, :].detach().cpu().numpy(), 'o', label = "POD Mode") 
ax[0].plot(A1_DT_Predict1.detach().cpu().numpy(), label = "SINDy : $\lambda = 10^{-2}$") 
ax[0].plot(A1_DT_Predict2.detach().cpu().numpy(), label = "SINDy - $\lambda_{matrix}$") 
ax[0].set_title('Mode 1')


ax[1].plot(A1A2A3_time_deriv[1, :].detach().cpu().numpy(), 'o', label = "POD Mode") 
ax[1].plot(A2_DT_Predict1.detach().cpu().numpy(), label = "SINDy - $\lambda = 10^{-2}$") 
ax[1].plot(A2_DT_Predict2.detach().cpu().numpy(), label = "SINDy - $\lambda_{matrix}$") 
ax[1].set_title('Mode 2')


ax[2].plot(A1A2A3_time_deriv[2, :].detach().cpu().numpy(), 'o', label = "POD Mode") 
ax[2].plot(A3_DT_Predict1.detach().cpu().numpy(), label = "SINDy : $\lambda = 10^{-2}$") 
ax[2].plot(A3_DT_Predict2.detach().cpu().numpy(), label = "SINDy : $\lambda_{matrix}$")
ax[2].legend()
ax[2].set_title('Mode 3')

fig.subplots_adjust(top=0.8)
plt.show()